In [9]:
# !pip install PyKomoran
# !pip install --upgrade PyKomoran
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# 텍스트 파일 불러오기
with open('text.txt', 'r', encoding='cp949') as file:
    text = file.read()

# 불용어 리스트 만들기
f=open('stopwords.txt','r',encoding='utf-8')
stop_words = f.read().splitlines()


In [3]:
len(text)

2344

In [4]:
# Initialize Komoran tokenizer
okt = Okt()
# Preprocessing function using KoNLPy2
def preprocess_text(text, stop_words):
    # Remove non-Korean characters and whitespace
    text = re.sub(r'[^가-힣\s]', '', text)
    # Tokenize the text using Komoran
    pos_tags = okt.pos(text)
    filtered_words = [word for word, pos in pos_tags if pos in ["Noun", "Adjective", "Verb"]]
    # Remove stopwords
    words = [word for word in filtered_words if word not in stop_words]
    return words


In [5]:
# TF-IDF 점수 계산하기
def compute_tfidf_scores_v2(text, stop_words, preprocess_func):
# text를 문장 단위로 분해하기
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    sentences = [s for s in sentences if len(s) > 0]
    
# 미리 정의한 전처리 함수를 통해 단어 단위로 토큰화 후 TF-IDF 계산 함수 정의
# x에는 문장이 들어감
    vectorizer = TfidfVectorizer(tokenizer=lambda x: preprocess_func(x, stop_words))
# 문장 단위로 TF-IDF 점수 계산 및 매트릭스에 저장
# 행이 문장, 열은 단어 - 즉, 특정 문장에 있는 특정 단어 별 tf-idf 점수가 값
# TF : 문장에서 단어  / IDF : 문단에서 문장
    tfidf_matrix = vectorizer.fit_transform(sentences)
# TF-IDF가 계산된 단어 목록
    feature_names = vectorizer.get_feature_names_out()
# TFIDF 매트릭스에서 같은 단어의 점수들을 모두 합산해서 도출   
    tfidf_scores = {}
    for word in feature_names:
        tfidf_scores[word] = np.sum(tfidf_matrix[:, vectorizer.vocabulary_[word]])
        
    return tfidf_scores

In [6]:
# 단어 그래프 생성
def word_graph_with_tfidf(words, tfidf_scores, graph):
    window_size = 3
    for idx, word in enumerate(words):
        if word not in graph:
            graph.add_node(word, weight=tfidf_scores.get(word, 1))  # Using TF-IDF score as weight
        for j in range(idx+1, idx+window_size):
            if j < len(words):
                if words[j] not in graph:
                    graph.add_node(words[j], weight=tfidf_scores.get(words[j], 1))  # Using TF-IDF score as weight
                graph.add_edge(word, words[j])

In [11]:
# 텍스트 랭킹 함수
def text_ranking(graph, d=0.85, max_iter=50):
# 주어진 그래프를 행렬로 변환 각 요소의 값은 노드 간 가중치
    matrix = nx.to_numpy_matrix(graph)
# 노드 개수 계산
    num_of_words = graph.number_of_nodes()
# 모든 노드의 초기 랭크를 같게 함
    ranks = np.ones(num_of_words) / num_of_words
# text_rank 알고리즘 실행
    for _ in range(max_iter):
        new_ranks = (1-d) + d * matrix.T.dot(ranks.reshape(-1,1))
        if np.linalg.norm(new_ranks - ranks, 2) < 1e-8:
            break
        ranks = new_ranks
# 결과 반환
    word_ranks = {word: rank for word, rank in zip(graph.nodes(), ranks)}
    sorted_word_ranks = sorted(word_ranks.items(), key=lambda x: x[1], reverse=True)
    keywords = [word for word, rank in sorted_word_ranks]
    return keywords

In [12]:
# 그래프 생성
graph = nx.Graph()

words = preprocess_text(text, stop_words)

tfidf_scores = compute_tfidf_scores_v2(text, stop_words, preprocess_text)

word_graph_with_tfidf(words, tfidf_scores, graph)

keywords = text_ranking(graph)

In [18]:
print(len(keywords))

403


In [20]:
print(keywords[0:50])

['스', '사피', '인간', '인류', '출', '간', '책', '저자', '서문', '더', '특별', '이제', '할', '이해', '한다', '인공', '지능', '시대', '세상', '동료', '나아갈', '마음', '전하', '통찰', '명실', '역사', '인문', '가로지르는', '미래', '작', '호소', '풍부한', '없었다', '호모', '읽고', '있어', '시장', '제작', '추천', '지식', '리', '사랑', '통해', '수', '글', '종횡무진', '주년', '이후', '강조', '상부']


### 생각한대로 나오지는 않는다..